In [1]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install rdp

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install bs4

In [3]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [1]:
from bs4 import BeautifulSoup
import numpy as np

In [2]:
def getSoup(file):
  with open(file, "r") as infile:
    xml_string = infile.read()
  soup = BeautifulSoup(xml_string, "xml")
  #trace_strings = soup.find_all("trace")
  return soup

In [3]:
import numpy as np
from rdp import rdp
import math
EPSILON = 1e-12
MACHINE_EPSILON = 1.12e-16


class point:
  def __init__(self, x, y):
    self.x = x
    self.y = y
#note calculations may be off because of int vs float
def isMachineZero (val) : 
  return (val >= -MACHINE_EPSILON and val <= MACHINE_EPSILON)
def hypot (x, y) : 
  return math.sqrt(x * x + y * y)
#def point (x, y) : return ( Point(x,y) )
def pointLength (p) : 
  return hypot(p.x, p.y)
def pointNegate (p) : 
  return point(-p.x, -p.y)
def pointAdd (p1, p2) : 
  return point(p1.x + p2.x, p1.y + p2.y)
def pointSubtract (p1, p2) : 
  return point(p1.x - p2.x, p1.y - p2.y)
def pointMultiplyScalar (p, n) : 
  return point(p.x * n, p.y * n)
def pointDot (p1, p2) : 
  return p1.x * p2.x + p1.y * p2.y
def pointDistance (p1, p2) : 
  return hypot(p1.x - p2.x, p1.y - p2.y)
def pointNormalize (p, length = 1) : 
  return pointMultiplyScalar(p, (length / pointLength(p)) if pointLength(p) else 0) #POSSIBLE FLAW
class createSegment: 
  def __init__(self, p, i, o):
    self.p=p
    self.i=i
    self.o=o #POSSIBLE FLAW (CHANGED)

def chordLengthParameterize (points, first, last)  :
    u = [0]
    for i in range(first+1, last+1):
    #for (let i = first + 1 i <= last i++) :
        #u[i - first] = u[i - first - 1] + pointDistance(points[i], points[i - 1])
        u.append(u[i - first - 1] + pointDistance(points[i], points[i - 1]))
    
    for i in range(1, last-first+1):
    #for (let i = 1, m = last - first i <= m i++) :
        u[i] /= u[last-first]
    
    return u

# Use Newton-Raphson iteration to find better root.
def findRoot (curve, point, u) :
    curve1 = []
    curve2 = []
    # Generate control vertices for Q'
    for i in range(3) : #FLAG
        curve1.append(pointMultiplyScalar(pointSubtract(curve[i + 1], curve[i]), 3))
    
    # Generate control vertices for Q''
    for i in range(2):
    #for (let i = 0 i <= 1 i++) :
        curve2.append(pointMultiplyScalar(pointSubtract(curve1[i + 1], curve1[i]), 2))
    
    # Compute Q(u), Q'(u) and Q''(u)
    pt = evaluate(3, curve, u)
    pt1 = evaluate(2, curve1, u)
    pt2 = evaluate(1, curve2, u)
    diff = pointSubtract(pt, point)
    df = pointDot(pt1, pt1) + pointDot(diff, pt2)
    # u = u - f(u) / f'(u)
    #return isMachineZero(df) ? u : u - pointDot(diff, pt1) / df
    return (u if isMachineZero(df) else (u-pointDot(diff,pt1)/df))

# Evaluate a bezier curve at a particular parameter value
def evaluate (degree, curve, t) :
    # Copy array
    tmp = curve[:]#.slice() FLAG
    # Triangle computation
    for i in range(1, degree+1):
    #for (let i = 1 i <= degree i++) :
        for j in range(degree):
        #for (let j = 0 j <= degree - i j++) :
            tmp[j] = pointAdd(pointMultiplyScalar(tmp[j], 1 - t), pointMultiplyScalar(tmp[j + 1], t))

    return tmp[0]

def addCurve (segments, curve)  :
    prev = segments[len(segments) - 1]
    prev.o = pointSubtract(curve[1], curve[0])
    segments.append(createSegment(curve[3], pointSubtract(curve[2], curve[3]), None))

# Use least-squares method to find Bezier control points for region.
def generateBezier (points, first, last, uPrime, tan1, tan2)  :
    epsilon = EPSILON
    #abs = Math.abs
    pt1 = points[first]
    pt2 = points[last]
    # Create the C and X matrices
    C = [[0, 0],[0, 0]]
    X = [0, 0]
    for i in range(0, last-first+1):
    #for (let i = 0, l = last - first + 1 i < l i++) :
        u = uPrime[i] 
        t = 1 - u
        b = 3 * u * t
        b0 = t * t * t
        b1 = b * t
        b2 = b * u
        b3 = u * u * u
        a1 = pointNormalize(tan1, b1)
        a2 = pointNormalize(tan2, b2)
        tmp = pointSubtract(pointSubtract(points[first + i], pointMultiplyScalar(pt1, b0 + b1)), pointMultiplyScalar(pt2, b2 + b3))
        C[0][0] += pointDot(a1, a1)
        C[0][1] += pointDot(a1, a2)
        # C[1][0] += a1.dot(a2)
        C[1][0] = C[0][1]
        C[1][1] += pointDot(a2, a2)
        X[0] += pointDot(a1, tmp)
        X[1] += pointDot(a2, tmp)
    
    # Compute the determinants of C and X
    detC0C1 = C[0][0] * C[1][1] - C[1][0] * C[0][1]
    alpha1=None
    alpha2=None
    if (abs(detC0C1) > epsilon) :
        # Kramer's rule
        detC0X = C[0][0] * X[1] - C[1][0] * X[0]
        detXC1 = X[0] * C[1][1] - X[1] * C[0][1]
        # Derive alpha values
        alpha1 = detXC1 / detC0C1
        alpha2 = detC0X / detC0C1
    
    else :
        # Matrix is under-determined, try assuming alpha1 == alpha2
        c0 = C[0][0] + C[0][1]
        c1 = C[1][0] + C[1][1]
        #alpha1 = alpha2 = abs(c0) > epsilon ? X[0] / c0 : abs(c1) > epsilon ? X[1] / c1 : 0
        alpha1 = alpha2 = X[0] / c0 if abs(c0) > epsilon else X[1] / c1 if abs(c1) > epsilon else 0
    
    # If alpha negative, use the Wu/Barsky heuristic (see text)
    # (if alpha is 0, you get coincident control points that lead to
    # divide by zero in any subsequent NewtonRaphsonRootFind() call.
    segLength = pointDistance(pt2, pt1)
    eps = epsilon * segLength
    handle1 = None
    handle2 = None
    if (alpha1 < eps or alpha2 < eps) :
        # fall back on standard (probably inaccurate) formula,
        # and subdivide further if needed.
        alpha1 = alpha2 = segLength / 3
    
    else :
        # Check if the found control points are in the right order when
        # projected onto the line through pt1 and pt2.
        line = pointSubtract(pt2, pt1)
        # Control points 1 and 2 are positioned an alpha distance out
        # on the tangent vectors, left and right, respectively
        handle1 = pointNormalize(tan1, alpha1)
        handle2 = pointNormalize(tan2, alpha2)
        if (pointDot(handle1, line) - pointDot(handle2, line) > segLength * segLength) :
            # Fall back to the Wu/Barsky heuristic above.
            alpha1 = alpha2 = segLength / 3
            handle1 = handle2 = None # Force recalculation
        
    
    # First and last control points of the Bezier curve are
    # positioned exactly at the first and last data points
    return [pt1, pointAdd(pt1, handle1 or pointNormalize(tan1, alpha1)), pointAdd(pt2, handle2 or pointNormalize(tan2, alpha2)), pt2]

# Given set of points and their parameterization, try to find
# a better parameterization.
def reparameterize (points, first, last, u, curve) :
    #for (let i = first i <= last i++) :
    for i in range(first, last+1):
        u[i - first] = findRoot(curve, points[i], u[i - first])
    
    # Detect if the new parameterization has reordered the points.
    # In that case, we would fit the points of the path in the wrong order.
    #for (let i = 1, l = u.length i < l i++) :
    for i in range (1, len(u)):
        if (u[i] <= u[i - 1]):
            return False
    
    return True


class error:
  def __init__(self, error, index):
    self.error = error
    self.index = index
# Find the maximum squared distance of digitized points to fitted curve.
def findMaxError (points, first, last, curve, u)  :
    index = math.floor((last - first + 1) / 2)
    maxDist = 0
    #for (let i = first + 1 i < last i++) :
    for i in range (first+1, last):
        P = evaluate(3, curve, u[i - first])
        v = pointSubtract(P, points[i])
        dist = v.x * v.x + v.y * v.y # squared
        if (dist >= maxDist) :
            maxDist = dist
            index = i
        
    return error(maxDist, index)
    #return :
    #   error: maxDist,
    #    index: index,
    



def fit (points, closed, error) :
    #points=filterRepeats(points)
    #points=rdpFilter(points,0.3)
    length = len(points)
    if length == 0 and type(length) == int:
        return []
    segments = [createSegment(points[0], None, None)]
    if length == 1:
      return segments
    fitCubic(points, segments, error, 0, length - 1, 
    pointSubtract(points[1], points[0]), 
    pointSubtract(points[length - 2], points[length - 1]))
    if (closed) :
        segments.pop(0)
        segments.pop()
    
    return segments
#note MAY HAVE TO FIND ALTERNATIVE TO === instead of using ==
def fitCubic (points, segments, error, first, last, tan1, tan2):
    #  Use heuristic if region only has two points in it
    if (last - first == 1) :
        pt1 = points[first]
        pt2 = points[last]
        dist = pointDistance(pt1, pt2) / 3
        addCurve(segments, [pt1, pointAdd(pt1, pointNormalize(tan1, dist)), pointAdd(pt2, pointNormalize(tan2, dist)), pt2])
        return
    
    # Parameterize points, and attempt to fit curve
    uPrime = chordLengthParameterize(points, first, last)
    maxError = max(error, error * error)
    split=None
    parametersInOrder = True
    # Try not 4 but 5 iterations
    #for (let i = 0 i <= 4 i++) :
    for i in range(5):
        curve = generateBezier(points, first, last, uPrime, tan1, tan2)
        #  Find max deviation of points to fitted curve
        Max = findMaxError(points, first, last, curve, uPrime)
        if (Max.error < error and parametersInOrder) :
            addCurve(segments, curve)
            #for segment in segments:
            #  print((str(segment.i.x) if segment.i else "None") + " " + (str(segment.i.y) if segment.i else "None") + " " + (str(segment.o.x) if segment.o else "None") + " " + (str(segment.o.y) if segment.o else "None") + " " + (str(segment.p.x) if segment.p else "None") + " " + (str(segment.p.y) if segment.p else "None"))
            return
        
        split = Max.index
        # If error not too large, try reparameterization and iteration
        if (Max.error >= maxError):
            break
        parametersInOrder = reparameterize(points, first, last, uPrime, curve)
        maxError = Max.error
    
    # Fitting failed -- split at max error point and fit recursively
    tanCenter = pointSubtract(points[split - 1], points[split + 1])
    fitCubic(points, segments, error, first, split, tan1, tanCenter)
    fitCubic(points, segments, error, split, last, pointNegate(tanCenter), tan2)


def filterRepeats(points):
  filtered_points=[]
  for i in range(len(points)-1):
    if points[i].x != points[i+1].x or points[i].y != points[i+1].y:
      filtered_points.append(points[i])
  filtered_points.append(points[len(points)-1])
  return filtered_points
def rdpFilter(points, epsilon):
  arr=[]
  for i in range(len(points)):
    arr.append([points[i].x,points[i].y])
  return arrayToPoints(rdp(arr, epsilon=epsilon))


# Assign parameter values to digitized points
# using relative distances between points.

def svgPath (segments, closed, precision) :
    length = len(segments)
    precisionMultiplier = 10 ** precision
    def Round(n, precisionMultiplier):
      return ((round(n * precisionMultiplier) / precisionMultiplier) if precision < 16 else n)
    def formatPair (x, y):
      return (str(Round(x, precisionMultiplier)) + ',' + str(Round(y, precisionMultiplier)))
    first = True
    prevX = None
    prevY = None
    outX = None
    outY = None
    parts = []
    if length==1:
      #return [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]],[[segments[0].p.x,segments[0].p.y]]]
      return 'M' + str(segments[0].p.x) + "," + str(segments[0].p.y) #CHANGED
    
    if (not length):
        return ''
    for i in range(length):
    #for (let i = 0 i < length i++)
        #addSegment(segments[i])
        #if not(prevX==segments[i].p.x and prevY==segments[i].p.y):
        segmentData = []
        segment=segments[i]
        #repeat=False
        curX = segment.p.x
        curY = segment.p.y
        if (first) :
            parts.append('M' + formatPair(curX, curY))
        else :
            inX = curX + (segment.i.x if segment.i else 0)
            inY = curY + (segment.i.y if segment.i else 0)
            if (not (inX == curX and inY == curY and outX == prevX and outY == prevY)) :
                  # c = relative curveto:
                  #segmentData.extend([prevX, prevY, outX-prevX, outY-prevY, inX - prevX, inY - prevY, curX - prevX, curY - prevY])
                segmentData.extend([prevX, prevY, outX, outY, inX, inY, curX, curY])
                parts.append('c' +
                      formatPair(outX - prevX, outY - prevY) +
                      ' ' +
                      formatPair(inX - prevX, inY - prevY) +
                      ' ' +
                      formatPair(curX - prevX, curY - prevY))

        prevX = curX
        prevY = curY
        outX = curX + (segment.o.x if segment.o else 0)
        outY = curY + (segment.o.y if segment.o else 0)
        first = False

    # Close path by drawing first segment again
    #return [allSegmentData, endPoints]
    return ''.join(parts)

def getSegmentsPathData (segments, closed, precision)  :
    length = len(segments)
    precisionMultiplier = 10 ** precision
    def Round(n, precisionMultiplier):
      return ((round(n * precisionMultiplier) / precisionMultiplier) if precision < 16 else n)
    def formatPair (x, y):
      return (str(Round(x, precisionMultiplier)) + ',' + str(Round(y, precisionMultiplier)))
    first = True
    prevX = None
    prevY = None
    outX = None
    outY = None
    parts = []
    if length==1:
      return [[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]],[[segments[0].p.x,segments[0].p.y]]]
      #return 'M' + str(segments[0].p.x) + "," + str(segments[0].p.y) #CHANGED
    
    if (not length):
        return []
    allSegmentData=[]
    endPoints = []
    for i in range(length):
    #for (let i = 0 i < length i++)
        #addSegment(segments[i])
        #if not(prevX==segments[i].p.x and prevY==segments[i].p.y):
        segmentData = []
        segment=segments[i]
        #repeat=False
        curX = segment.p.x
        curY = segment.p.y
        if (first) :
            parts.append('M' + formatPair(curX, curY))
        else :
            inX = curX + (segment.i.x if segment.i else 0)
            inY = curY + (segment.i.y if segment.i else 0)
            if (not (inX == curX and inY == curY and outX == prevX and outY == prevY)) :
                  # c = relative curveto:
                  #segmentData.extend([prevX, prevY, outX-prevX, outY-prevY, inX - prevX, inY - prevY, curX - prevX, curY - prevY])
                segmentData.extend([prevX, prevY, outX, outY, inX, inY, curX, curY])
                parts.append('c' +
                      formatPair(outX - prevX, outY - prevY) +
                      ' ' +
                      formatPair(inX - prevX, inY - prevY) +
                      ' ' +
                      formatPair(curX - prevX, curY - prevY))

                features = [segmentData[6]-segmentData[0], 
                            segmentData[7]-segmentData[1], 
                            math.sqrt((segmentData[0]-segmentData[2])**2+(segmentData[1]-segmentData[3])**2), 
                            math.sqrt((segmentData[6]-segmentData[4])**2+(segmentData[7]-segmentData[5])**2), 
                            math.atan2(segmentData[3]-segmentData[1], segmentData[2]-segmentData[0])-math.atan2(segmentData[7]-segmentData[1], segmentData[6]-segmentData[0]),
                            math.atan2(segmentData[5]-segmentData[7], segmentData[4]-segmentData[6])-math.atan2(segmentData[1]-segmentData[7], segmentData[0]-segmentData[6]), 1]
                allSegmentData.append(features)
                endPoints.append([segmentData[6],segmentData[7]])
            #else:
              #print("bruh what")

        prevX = curX
        prevY = curY
        outX = curX + (segment.o.x if segment.o else 0)
        outY = curY + (segment.o.y if segment.o else 0)
        first = False

    # Close path by drawing first segment again
    return [allSegmentData, endPoints]
    #return ''.join(parts)

class options:
  def __init__(self, closed, tolerance, precision):
    self.closed = closed
    self.tolerance = tolerance
    self.precision=precision

def f1(p):
  return point(p.x, p.y)

def f2(p):
  return point(p[0], p[1])

#def simplify(points, options) :
#  return fit(list(map((f1 if (type(points[0].x) == int or type(points[0].x) == float) else f2), points)), options.closed, options.tolerance if options.tolerance else 2.5)
#def getFeatures (points, options):

def getSvgPath (points, options) :
    if (len(points) == 0 and type(len(points))==int) :
        return ''
    return svgPath(fit(list(map(f1 if (type(points[0].x) == int or type(points[0].x) == float) else f2, points)), options.closed, options.tolerance if options.tolerance else 2.5), options.closed, options.precision if options.precision else 5)

def getFeatures (points, options)  :
    if (len(points) == 0 and type(len(points))==int) :
        return []
    return getSegmentsPathData(fit(list(map(f1 if (type(points[0].x) == int or type(points[0].x) == float) else f2, points)), options.closed, options.tolerance if options.tolerance else 2.5), options.closed, options.precision if options.precision else 5)
    #return getSegmentsPathData(fit(points.map(typeof points[0].x == 'number' ? (p) => point(p.x, p.y) : (p) => point(p[0], p[1])), options.closed, options.tolerance ?? 2.5), options.closed, options.precision ?? 5)

In [12]:
def testFit (arr) :
  points = arrayToPoints(arr)
  for segment in fit(list(map((f1 if (type(points[0].x) == int or type(points[0].x) == float) else f2), points)), False, 15):
    print((str(segment.i.x) if segment.i else "None") + " " + (str(segment.i.y) if segment.i else "None") + " " + (str(segment.o.x) if segment.o else "None") + " " + (str(segment.o.y) if segment.o else "None") + " " + (str(segment.p.x) if segment.p else "None") + " " + (str(segment.p.y) if segment.p else "None"))

In [4]:
def arrayToPoints (arr) :
  points = []
  for i in arr:
    points.append(point(i[0], i[1]))
  return(points)

In [14]:
getSoup("C:/Users/aiden/Downloads/ProjectData/LGINKML/INKMLs/2012/001-equation000.inkml")

<?xml version="1.0" encoding="utf-8"?>
<ink xmlns="http://www.w3.org/2003/InkML">
<traceFormat>
<channel name="X" type="decimal"/>
<channel name="Y" type="decimal"/>
</traceFormat>
<annotation type="truth">  $y = Ax + A^2$</annotation>
<annotation type="UI">2012_IVC_CROHME_F01_E000</annotation>
<annotation type="copyright">LUNAM/IRCCyN</annotation>
<annotation type="writer">CROHME01</annotation>
<annotationXML encoding="Content-MathML" type="truth">
<math xmlns="http://www.w3.org/1998/Math/MathML">
<mrow>
<mi xml:id="y_1">y</mi>
<mrow>
<mo xml:id="=_1">=</mo>
<mrow>
<mi xml:id="A_1">A</mi>
<mrow>
<mi xml:id="x_1">x</mi>
<mrow>
<mo xml:id="+_1">+</mo>
<msup>
<mi xml:id="A_2">A</mi>
<mn xml:id="2_1">2</mn>
</msup>
</mrow>
</mrow>
</mrow>
</mrow>
</mrow>
</math>
</annotationXML>
<trace id="0">
1.0066 5.46535, 1.0066 5.4645, 1.00617 5.46322, 1.00617 5.46237, 1.00575 5.46152, 1.00575 5.46109, 1.00575 5.46152, 1.00532 5.46109, 1.00489 5.46109, 1.00447 5.46152, 1.00404 5.46237, 1.00319 5.4653

In [15]:
trace_split = "9888 3069, 9888 3069, 9888 3069, 9879 3056, 9868 3042, 9868 3042, 9849 3024, 9849 3024, 9822 3007, 9805 3006, 9781 3007, 9757 3021, 9727 3044, 9695 3084, 9662 3134, 9630 3196, 9603 3264, 9582 3335, 9571 3405, 9573 3473, 9582 3532, 9603 3577, 9635 3612, 9672 3635, 9721 3654, 9773 3662, 9825 3660, 9879 3651, 9927 3630"
if trace_split[0]=="\n":
    trace_split=trace_split[1:]
if trace_split[-1]=="\n":
    trace_split=trace_split[:-1]
print(trace_split)

9888 3069, 9888 3069, 9888 3069, 9879 3056, 9868 3042, 9868 3042, 9849 3024, 9849 3024, 9822 3007, 9805 3006, 9781 3007, 9757 3021, 9727 3044, 9695 3084, 9662 3134, 9630 3196, 9603 3264, 9582 3335, 9571 3405, 9573 3473, 9582 3532, 9603 3577, 9635 3612, 9672 3635, 9721 3654, 9773 3662, 9825 3660, 9879 3651, 9927 3630


In [5]:
def _parse_one_trace(trace_string):
  '''if len(trace_string.split("\n"))==3:
    trace_split = trace_string.split("\n")[1].split(",")
  else:
    trace_split = trace_string.split(">")
  try:
    trace_split = trace_string.split("\n")[1].split(",")
  except:
    print(trace_string.split("\n"))'''
  trace_split=trace_string.string
  if trace_split[0]=="\n":
    trace_split=trace_split[1:]
  if trace_split[-1]=="\n":
    trace_split=trace_split[:-1]
  trace_split=trace_split.split(",")
  a = [[float(s) for s in entry.strip().split(" ")] for entry in trace_split]#changed from int to float
  #a = [np.array([int(s) for s in entry.strip().split(" ")], dtype=np.int32) for entry in trace_split]
  #a = np.array(a + [np.array([0, 0], dtype=np.int32)] * (max_length-len(a)))
  return a


def getSvg(file, tolerance):
  with open(file, "r") as infile:
    xml_string = infile.read()
  soup = BeautifulSoup(xml_string, "xml")
  trace_strings = soup.find_all("trace")
  a=""
  first = True
  prev = None
  for trace_string in trace_strings:
    path=getSvgPath(arrayToPoints(_parse_one_trace(trace_string)), options(False,tolerance,0))
    #if not first:
      #print(path)

      #print(path)
      #dx=path[1][len(path[1])-1][0] - prev[1][len(prev[1])-1][0]
      #dy=path[1][len(path[1])-1][1]-prev[1][len(prev[1])-1][1]
      #a += [[dx, dy, math.sqrt(dx**2/9+dy**2/9), math.sqrt(dx**2/9+dy**2/9), 0.0, 0.0, 0]]
    #a += path[0]
    a+=path
    #print(path)
    #a += 
  #a = np.array([_parse_one_trace(str(trace_string), maxLen) for trace_string in trace_strings])
  return a #+ [0,0] * (maxLen-len(a))

def parse_traces(file, tolerance):
  with open(file, "r") as infile:
    xml_string = infile.read()
  soup = BeautifulSoup(xml_string, "xml")
  trace_strings = soup.find_all("trace")
  a=[]
  first = True
  prev = None
  for trace_string in trace_strings:
    #print("parseonetrace:")
    #print(arrayToPoints(_parse_one_trace(trace_string)))
    path=getFeatures(arrayToPoints(_parse_one_trace(trace_string)), options(False,tolerance,0))
    if not first:
      #print(path)

      #print(path)
      try:
        dx=path[1][len(path[1])-1][0] - prev[1][len(prev[1])-1][0]
        dy=path[1][len(path[1])-1][1]-prev[1][len(prev[1])-1][1]
      except:
        print("path:")
        print(path)
        print("trace_string:")
        print(trace_string.string)
        print("file:")
        print(file)
      a += [[dx, dy, math.sqrt(dx**2/9+dy**2/9), math.sqrt(dx**2/9+dy**2/9), 0.0, 0.0, 0]]
    a += path[0]
    prev = path[:]
    first=False
    #print(path)
    #a += 
  #a = np.array([_parse_one_trace(str(trace_string), maxLen) for trace_string in trace_strings])
  return a #+ [0,0] * (maxLen-len(a))

'''def _parse_one_trace(trace_string):
  trace_split = trace_string.split("\n")[1].split(",")
  return [tuple([int(s) for s in entry.strip().split(" ")]) for entry in trace_split]

def parse_traces(file):
  with open(file, "r") as infile:
    xml_string = infile.read()
  soup = BeautifulSoup(xml_string, "xml")
  trace_strings = soup.find_all("trace")
  return [_parse_one_trace(str(trace_string)) for trace_string in trace_strings]'''

'def _parse_one_trace(trace_string):\n  trace_split = trace_string.split("\n")[1].split(",")\n  return [tuple([int(s) for s in entry.strip().split(" ")]) for entry in trace_split]\n\ndef parse_traces(file):\n  with open(file, "r") as infile:\n    xml_string = infile.read()\n  soup = BeautifulSoup(xml_string, "xml")\n  trace_strings = soup.find_all("trace")\n  return [_parse_one_trace(str(trace_string)) for trace_string in trace_strings]'

In [74]:
len(parse_traces("C:/Users/aiden/Downloads/ProjectData/LGINKML/INKMLs/2014train/81_user0.inkml", 20))

115

In [6]:
'''def _parse_one_character(group):
  character = group.annotation.string
  stroke_ids = [view.get("traceDataRef") for view in group.find_all("traceView")]

  return (character, stroke_ids)

# todo: Add something to return the sequence of unique identifiers ('x_1' etc.)
def parse_symbols_and_alignments(file):
  with open(file, "r") as infile:
    xml_string = infile.read()
  tree = BeautifulSoup(xml_string, "xml")
  trace_groups = tree.find_all("traceGroup")[1:]
  return [_parse_one_character(group) for group in trace_groups]'''

from collections import defaultdict
# todo: Add something to return the sequence of unique identifiers ('x_1' etc.)
def parse_symbols_and_alignments(file):
  with open(file, "r") as f:
    string = f.read()
  #lg=[]
  rel=0 #BOOLEAN sorta
  objects=[]
  rels=defaultdict(lambda: "NoRel")
  for part in string.split("\n"):
    line=part.split(", ")
    if (line[0]=="# Relations from SRT:"):
      rel=1
    else:
      if (line[0]!=''):
        if rel:
          #if rel > 1:
          rels[(line[1], line[2])]=line[3]
          #rel+=1
          #prev=line[1]
        else:
          if (line[0][0]=="#"):
            continue
          del line[3]
          objects.append((line[1:3], [int(l) for l in line[3:]]))
  def key(obj):
    return min(obj[1])
  objects=sorted(objects, key=key)
  output=[]

  output.append(objects[0][0][1])
  prev=objects[0][0][0]
  #print(rels)
  for ob in objects[1:]:
    #print((ob[0][0], prev))
    output.append(rels[(prev, ob[0][0])])
    output.append(ob[0][1])
    prev=ob[0][0]
    #output.append(ob[0][1])
    #output.append(rels[ob[0][0]])
  '''try:
    output.append(objects[-1][0][1])
  except:
    print(file)'''
  #return(objects, rels)
  return(output)



In [63]:
getFeatures("C:/Users/aiden/Downloads/ProjectData/LGINKML/INKMLs/2014train/81_user0.inkml", 20)

AttributeError: 'str' object has no attribute 'x'

In [73]:
getSvg("C:/Users/aiden/Downloads/ProjectData/LGINKML/INKMLs/2014train/81_user0.inkml", 20)

'M117.0,125.0c-5.15278,5.15278 -21.96762,48.96762 -21.0,48.0M118.0,124.0c10.23405,10.23405 10.12082,27.06941 14.0,40.0c0.92485,3.08285 4.87849,11.12151 3.0,13.0M102.0,159.0c-13.94705,-6.97353 32.90416,-0.09584 30.0,-3.0M144.0,173.0c-6.28086,-6.28086 6.66996,-11.16502 11.0,-9.0c2.9449,1.47245 2.01189,6.97622 1.0,9.0c-6.74966,13.49932 -22.19391,13.76122 -1.0,18.0c2.31125,0.46225 4.64298,1.0 7.0,1.0M170.0,164.0c0.0,9.0 0.0,18.0 0.0,27.0c0.0,1.79505 1.19723,-3.39445 2.0,-5.0c2.39455,-4.78909 14.0,-24.24896 14.0,-17.0M176.0,182.0c4.72016,2.8321 11.54335,8.0 16.0,8.0M207.0,151.0c9.21226,0.0 18.03175,-2.0 27.0,-2.0M210.0,163.0c8.98622,0.59908 17.99383,1.0 27.0,1.0M417.0,88.0c0.0,0.94281 -2.16865,-1.0724 -2.0,-2.0c1.7111,-9.41104 17.22641,-14.32078 20.0,-6.0c4.27455,12.82366 -27.0,27.23502 -27.0,36.0c0.0,4.61168 9.91023,0.28089 13.0,0.0c3.85651,-0.35059 20.7422,3.7422 17.0,0.0M462.0,77.0c2.1535,17.94587 4.0,25.58836 4.0,42.0M454.0,92.0c-6.94628,-1.73657 20.34976,-30.3128 35.0,-12.0c9.36361,11.

In [53]:
parse_symbols_and_alignments("C:/Users/aiden/Downloads/ProjectData/LGINKML/LGs/2014train/81_user0.lg")

('2_1', 'A_1')
('k_1', '2_1')
('=_1', 'k_1')
('2_2', '=_1')
('R_1', '2_2')
('A_2', 'R_1')
('k_2', 'A_2')
('2_3', 'k_2')
('R_2', '2_3')
('+_1', 'R_2')
('4_1', '+_1')
('R_3', '4_1')
('2_4', 'R_3')
('+_2', '2_4')
('A_3', '+_2')
('2_5', 'A_3')
('k_3', '2_5')
('_2', 'k_3')
('_1', '_2')


['A',
 'Sub',
 '2',
 'Right',
 'k',
 'NoRel',
 '=',
 'NoRel',
 '2',
 'Right',
 'R',
 'Right',
 'A',
 'Sub',
 'k',
 'NoRel',
 '2',
 'Right',
 'R',
 'Right',
 '+',
 'NoRel',
 '4',
 'Right',
 'R',
 'Sup',
 '2',
 'NoRel',
 '+',
 'Right',
 'A',
 'Sup',
 '2',
 'NoRel',
 'k',
 'NoRel',
 '\\sqrt',
 'NoRel',
 '-']

In [10]:
getFeatures(arrayToPoints([[10306, 1213], [10306, 1213], [10299, 1248], [10299, 1248], [10299, 1248], [10299, 1248], [10299, 1248], [10299, 1248]]), options(False, 20, 0))

[[], []]

In [7]:
import pickle as pkl
import os
years=["2012", "2013", "2014train", "2014test", "2016", "2019"]
#years = ["2019"]
#train=False
#name="LG"
#datapath="C:/Users/aiden/Downloads/ProjectData/"+year+"/"+ ("Train/" if train else "Test/") + name
X=[]
Y=[]
count=0
vocab={"Right" : 0, "Sub" : 1, "Sup" : 2, "Above" : 3, "Below" : 4, "Inside" : 5, "NoRel" : 6}
for year in years:
    directory="C:/Users/aiden/Downloads/ProjectData/LGINKML/INKMLs/"+year+"/"
    for filename in os.listdir(directory):
        #the filename unincluding .inkml
        lgpath="C:/Users/aiden/Downloads/ProjectData/LGINKML/LGs/"+year+"/"+filename[:len(filename)-6]+".lg"
        if os.path.exists(lgpath):
            X.append(parse_traces(directory+filename, tolerance=15))
            LG=parse_symbols_and_alignments(lgpath)
            for symbol in LG:
                if symbol not in vocab:
                    vocab[symbol]=len(vocab)
            Y.append(parse_symbols_and_alignments(lgpath))
        else:
            print("missing:")
            print(lgpath)
        count += 1

        if count % 100 == 0: print(count)
datapath="C:/Users/aiden/Downloads/ProjectData/FeatureData/"
with open(datapath+"X_Y_Vocab", "wb") as f:
    pkl.dump((X,Y,vocab), f)

100
200
300
path:
[[], []]
trace_string:
10306 1213, 10306 1213, 10299 1248, 10299 1248, 10299 1248, 10299 1248, 10299 1248, 10299 1248
file:
C:/Users/aiden/Downloads/ProjectData/LGINKML/INKMLs/2012/KME2G3_2_sub_52.inkml
path:
[[[0.0, 20.0, 0.0, 6.6666666666667425, -1.5707963267948966, 0.0, 1], [0.0, 27.0, 9.0, 9.0, 0.0, 0.0, 1], [-20.0, 163.0, 56.35631855904671, 56.3047669288471, -0.1220891373972981, 0.01679587812290495, 1], [-7.0, 38.0, 12.879786057574542, 12.879786057574508, -0.043283294952512597, -0.03115296755135133, 1], [0.0, 8.0, 2.6666666666665795, 2.6666666666667425, 0.1510153429214034, 0.0, 1], [0.0, 8.0, 2.6666666666667425, 2.6666666666665484, 0.0, 3.395266749728237, 1], [33.0, -82.0, 29.4637253433966, 29.463725343396394, -0.12893311821917175, 0.030236690584775516, 1], [113.0, -174.0, 89.07065127373095, 90.13074362047458, -0.16312719362574168, 0.215274951383015, 1], [99.0, -54.0, 21.530870099961174, 37.6000248944223, -0.28020355516343637, -5.261435153782926, 1], [-7.0, 139.0

KeyboardInterrupt: 

In [ ]:
with open(datapath+"X_Y_Vocab", "rb") as f:
    X, Y, vocab = pkl.load(f)

In [ ]:
length = len(X)
vocab_size = len(vocab)
X_train = []
Y_train = []
X_test = []
Y_test = []
for i in range(length):
    if (i%10):
        X_train.append(X[i])
        Y_train.append(Y[i])
    else:
        X_test.append(X[i])
        Y_test.append(Y[i])


In [ ]:
datapath="C:/Users/aiden/Downloads/ProjectData/FeatureData/"

In [ ]:
with open(datapath+"Train", "wb") as f:
    pkl.dump((X_train, Y_train, vocab), f)
with open(datapath+"Test", "wb") as f:
    pkl.dump((X_test, Y_test, vocab), f)

In [ ]:
print(len(X_train))
print(len(X_test))

11988
1333
